In this tutorial, we are going to demonstrate some basic tasks in graph learning. In general, many of the graph learning problems can fall into the following categories:

* Node classification: assign a label to a node.
* Link prediction: predict the existence of an edge between two nodes.
* Graph classification: assign a label to a graph.

Many real-world applications can be formulated as one of these graph problems.
* Fraud detection in financial transactions: transactions form a graph, where users are nodes and transactions are edges. In this case, we want to detect malicious users, which is to assign binary labels to users.
* Community detection in a social network: a social network is naturally a graph, where nodes are users and edges are interactions between users. We want to predict which community a node belongs to.
* Recommendation: users and items form a bipartite graph. They are connected with edges when users purchase items. Given users' purchase history, we want to predict what items a user will purchase in a near future. Thus, recommendation is a link prediction problem.
* Drug discovery: a molecule is a graph whose nodes are atoms. We want to predict the property of a molecule. In this case, we want to assign a label to a graph.

## Get started

DGL can be used with different deep learning frameworks. Currently, DGL can be used with Pytorch and MXNet. Here, we show how DGL works with Pytorch.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

When we load DGL, we need to set the DGL backend for one of the deep learning frameworks. Because this tutorial develops models in Pytorch, we have to set the DGL backend to Pytorch.

In [ ]:
import dgl
from dgl import DGLGraph

# Load Pytorch as backend
dgl.load_backend('pytorch')

Load the rest of the necessary libraries.

In [ ]:
import numpy as np

## GNN model

Typically, GNN is used to compute meaningful node embeddings. With the embeddings, we can perform many downstream tasks.

DGL provides two ways of implementing a GNN model:
* using the [nn module](https://doc.dgl.ai/features/nn.html), which contains many commonly used GNN modules.
* using the message passing interface to implement a GNN model from scratch.

For simplicity, we implement the GNN model in the tutorial with the nn module.

In this tutorial, we use [GraphSage](https://cs.stanford.edu/people/jure/pubs/graphsage-nips17.pdf), one of the first inductive GNN models. GraphSage performs the following computation on every node $v$ in the graph:

$$h_{N(v)}^{(l)} \gets AGGREGATE_k({h_u^{(l-1)}, \forall u \in N(v)})$$
$$h_v^{(l)} \gets \sigma(W^k \cdot CONCAT(h_v^{(l-1)}, h_{N(v)}^{(l)})),$$

where $N(v)$ is the neighborhood of node $v$ and $l$ is the layer Id.

The GraphSage model has multiple layers. In each layer, a vertex accesses its direct neighbors. When we stack $k$ layers in a model, a node $v$ access neighbors within $k$ hops. The output of the GraphSage model is node embeddings that represent the nodes and all information in the k-hop neighborhood.

<img src="https://github.com/zheng-da/DGL_devday_tutorial/raw/master/GNN.png" alt="drawing" width="600"/>

Use DGL's `nn` module to build the GraphSage model. `SAGEConv` implements the operations of `GraphSage` in a layer.

In [ ]:
from dgl.nn.pytorch import conv as dgl_conv

class GraphSAGEModel(nn.Module):
    def __init__(self,
                 in_feats,
                 n_hidden,
                 out_dim,
                 n_layers,
                 activation,
                 dropout,
                 aggregator_type):
        super(GraphSAGEModel, self).__init__()
        self.layers = nn.ModuleList()

        # input layer
        self.layers.append(dgl_conv.SAGEConv(in_feats, n_hidden, aggregator_type,
                                         feat_drop=dropout, activation=activation))
        # hidden layers
        for i in range(n_layers - 1):
            self.layers.append(dgl_conv.SAGEConv(n_hidden, n_hidden, aggregator_type,
                                             feat_drop=dropout, activation=activation))
        # output layer
        self.layers.append(dgl_conv.SAGEConv(n_hidden, out_dim, aggregator_type,
                                         feat_drop=dropout, activation=None))

    def forward(self, g, features):
        h = features
        for layer in self.layers:
            h = layer(g, h)
        return h

Interested readers can check out our [online tutorials](https://doc.dgl.ai/tutorials/models/index.html) to see how to use DGL's message passing interface to implement GNN models.

## Prepare the dataset for the tutorial

DGL has a large collection of built-in datasets. Please see [this doc](https://doc.dgl.ai/api/python/data.html) for more information.

In this tutorial, we use a citation network called pubmed for demonstration. A node in the citation network is a paper and an edge represents the citation between two papers. This dataset has 19,717 papers and 88,651 citations. Each paper has a sparse bag-of-words feature vector and a class label.

All other graph data, such as node features, are stored as NumPy tensors. When we load the tensors, we convert them to Pytorch tensors.

In [ ]:
from dgl.data import citegrh

# load and preprocess the pubmed dataset
data = citegrh.load_pubmed()

# sparse bag-of-words features of papers
features = torch.FloatTensor(data.features)
# the number of input node features
in_feats = features.shape[1]
# class labels of papers
labels = torch.LongTensor(data.labels)
# the number of unique classes on the nodes.
n_classes = data.num_labels

For small datasets, DGL stores the network structure in a [NetworkX](https://networkx.github.io) object. NetworkX is a very popular Python graph library. It provides comprehensive API for graph manipulation and is very useful for preprocessing small graphs.

Here we use NetworkX to remove all self-loops in the graph.

In [ ]:
data.graph.remove_edges_from(data.graph.selfloop_edges())

Then we create a DGLGraph from the grpah dataset and convert it to a read-only DGLGraph, which supports more efficient computation. Currently, DGL sampling API only works on read-only DGLGraphs.

In [ ]:
g = DGLGraph(data.graph)
g.readonly()

## Node classification in the semi-supervised setting

Let us perform node classification in a semi-supervised setting. In this setting, we have the entire graph structure and all node features. We only have labels on some of the nodes. We want to predict the labels on other nodes. Even though some of the nodes do not have labels, they connect with nodes with labels. Thus, we train the model with both labeled nodes and unlabeled nodes. Semi-supervised learning can usually improve performance.

<img src="https://github.com/zheng-da/DGL_devday_tutorial/raw/master/node_classify1.png" alt="drawing" width="200"/>

This dependency graph shows a better view of how labeled and unlabled nodes are used in the training.
<img src="https://github.com/zheng-da/DGL_devday_tutorial/raw/master/node_classify2.png" alt="drawing" width="800"/>

First, we create a 2-layer GraphSage model.

In [ ]:
# Hyperparameters
n_hidden = 64
n_layers = 2
dropout = 0.5
aggregator_type = 'gcn'

gconv_model = GraphSAGEModel(in_feats,
                             n_hidden,
                             n_classes,
                             n_layers,
                             F.relu,
                             dropout,
                             aggregator_type)

Now we create the node classification model based on the GraphSage model. The GraphSage model takes a DGLGraph object and node features as input and computes node embeddings as output. With node embeddings, we use a cross entropy loss to train the node classification model.

Now we create the node classification model based on the GraphSage model. The GraphSage model takes a DGLGraph object and node features as input and computes node embeddings as output. With node embeddings, we use a cross entropy loss to train the node classification model.

In [ ]:
class NodeClassification(nn.Module):
    def __init__(self, gconv_model, n_hidden, n_classes):
        super(NodeClassification, self).__init__()
        self.gconv_model = gconv_model
        self.loss_fcn = torch.nn.CrossEntropyLoss()

    def forward(self, g, features, train_mask):
        logits = self.gconv_model(g, features)
        return self.loss_fcn(logits[train_mask], labels[train_mask])

After defining a model for node classification, we need to define an evaluation function to evaluate the performance of a trained model.

In [ ]:
def NCEvaluate(model, g, features, labels, test_mask):
    model.eval()
    with torch.no_grad():
        # compute embeddings with GNN
        logits = model.gconv_model(g, features)
        logits = logits[test_mask]
        test_labels = labels[test_mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == test_labels)
        return correct.item() * 1.0 / len(test_labels)

Prepare data for semi-supervised node classification

In [ ]:
# the dataset is split into training set, validation set and testing set.
train_mask = torch.BoolTensor(data.train_mask)
val_mask = torch.BoolTensor(data.val_mask)
test_mask = torch.BoolTensor(data.test_mask)

print("""----Data statistics------'
      #Classes %d
      #Train samples %d
      #Val samples %d
      #Test samples %d""" %
          (n_classes,
           data.train_mask.sum().item(),
           data.val_mask.sum().item(),
           data.test_mask.sum().item()))

After defining the model and evaluation function, we can put everything into the training loop to train the model.

In [ ]:
# Node classification task
model = NodeClassification(gconv_model, n_hidden, n_classes)

# Training hyperparameters
weight_decay = 5e-4
n_epochs = 150
lr = 1e-3

# create the Adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

dur = []
for epoch in range(n_epochs):
    # Set the model in the training mode.
    model.train()
    # forward
    loss = model(g, features, train_mask)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    acc = NCEvaluate(model, g, features, labels, val_mask)
    print("Epoch {:05d} | Loss {:.4f} | Accuracy {:.4f}"
          .format(epoch, loss.item(), acc))

print()
acc = NCEvaluate(model, g, features, labels, test_mask)
print("Test Accuracy {:.4f}".format(acc))

## Link prediction

The task of link prediction is to predict the existence of an edge between two nodes.
<img src="https://github.com/zheng-da/DGL_devday_tutorial/raw/master/link_predict1.png" alt="drawing" width="500"/>

In general, we consider that an edge connects two similar nodes. Thus, link prediction is to find pairs of similar nodes.

Traditional methods, such as SVD, only takes the graph structure into consideration for link prediction. GNN-based models can take both the graph structure and node features into consideration when estimating if two nodes are connected. Thus, GNN-based models can potentially generate better results.
<img src="https://github.com/zheng-da/DGL_devday_tutorial/raw/master/link_predict2.png" alt="drawing" width="300"/>

Like the node classification task, we first use GraphSage as the base model to compute node embeddings in this task as well. Similarly, we can replace GraphSage with many other GNN models to compute node embeddings.

In [ ]:
#Model hyperparameters
n_hidden = in_feats
n_layers = 1
dropout = 0.5
aggregator_type = 'gcn'

# create GraphSAGE model
gconv_model = GraphSAGEModel(in_feats,
                             n_hidden,
                             n_hidden,
                             n_layers,
                             F.relu,
                             dropout,
                             aggregator_type)

Unlike node classification, which uses node labels as the training supervision, link prediction simply uses connectivity of nodes as the training signal: nodes connected by edges are similar, while nodes not connected by edges are dissimilar.

To train such a model, we need to deploy negative sampling to construct negative samples. In the context of link prediction, a positive sample is an edge that exist in the original graph, while a negative sample is a pair of nodes that don't have an edge between them in the graph. We usually train on each positive sample with multiple negative samples.

After having the node embeddings, we compute the similarity scores on positive samples and negative samples. We construct the following loss function on a positive sample and the corresponding negative samples:

$$L = -log(\sigma(z_u^T z_v)) - Q \cdot E_{v_n \sim P_n(v)}(log(\sigma(-z_u^T z_{v_n}))),$$

where $Q$ is the number of negative samples.

With this loss, training should increase the similarity scores on the positive samples and decrease the similarity scores on negative samples.

In [ ]:
# NCE loss
def NCE_loss(pos_score, neg_score, neg_sample_size):
    pos_score = F.logsigmoid(pos_score)
    neg_score = F.logsigmoid(-neg_score).reshape(-1, neg_sample_size)
    return -pos_score - torch.sum(neg_score, dim=1)

class LinkPrediction(nn.Module):
    def __init__(self, gconv_model):
        super(LinkPrediction, self).__init__()
        self.gconv_model = gconv_model

    def forward(self, g, features, neg_sample_size):
        emb = self.gconv_model(g, features)
        pos_g, neg_g = edge_sampler(g, neg_sample_size, return_false_neg=False)
        pos_score = score_func(pos_g, emb)
        neg_score = score_func(neg_g, emb)
        return torch.mean(NCE_loss(pos_score, neg_score, neg_sample_size))

DGL provides an edge sampler `EdgeSampler`, which selects positive edge samples and negative edge samples efficiently. Thus, we can use it to generate positive samples and negative samples for link prediction. `EdgeSampler` generates `neg_sample_size` negative edges by corrupting the head or the tail node of a positive edge with some randomly sampled nodes.

<img src="https://github.com/zheng-da/DGL_devday_tutorial/raw/master/negative_edges.png" alt="drawing" width="400"/>

`edge_sampler` samples one tenth of the edges in the graph as positive edges. It returns a positive subgraph and a negative subgraph. The positive subgraph contains all positive edges sampled from the graph `g`, while the negative subgraph contains all negative edges constructed by the edge sampler.

In [ ]:
def edge_sampler(g, neg_sample_size, edges=None, return_false_neg=True):
    sampler = dgl.contrib.sampling.EdgeSampler(g, batch_size=int(g.number_of_edges()/10),
                                               seed_edges=edges,
                                               neg_sample_size=neg_sample_size,
                                               negative_mode='tail',
                                               shuffle=True,
                                               return_false_neg=return_false_neg)
    sampler = iter(sampler)
    return next(sampler)

After having the positive edge subgraph and the negative edge subgraph, we can now compute the similarity on the positive edge samples and negative edge samples.

In this tutorial, we use cosine similarity to measure the similarity between two nodes.

In [ ]:
def score_func(g, emb):
    src_nid, dst_nid = g.all_edges(order='eid')
    # Get the node Ids in the parent graph.
    src_nid = g.parent_nid[src_nid]
    dst_nid = g.parent_nid[dst_nid]
    # Read the node embeddings of the source nodes and destination nodes.
    pos_heads = emb[src_nid]
    pos_tails = emb[dst_nid]
    # cosine similarity
    return torch.sum(pos_heads * pos_tails, dim=1)

Like node classification, we define an evaluation function to evaluate the training result. Ideally, the similarity score of a positive edge should be higher than all negative edges.

We evaluate the performance of link prediction with MRR (mean reciprocal rank):
$$MRR=\frac{1}{|Q|} \Sigma_{i=1}^{|Q|} \frac{1}{rank_i}.$$

In [ ]:
def LPEvaluate(gconv_model, g, features, eval_eids, neg_sample_size):
    gconv_model.eval()
    with torch.no_grad():
        emb = gconv_model(g, features)
        
        pos_g, neg_g = edge_sampler(g, neg_sample_size, eval_eids, return_false_neg=True)
        pos_score = score_func(pos_g, emb)
        neg_score = score_func(neg_g, emb).reshape(-1, neg_sample_size)
        filter_bias = neg_g.edata['false_neg'].reshape(-1, neg_sample_size)

        pos_score = F.logsigmoid(pos_score)
        neg_score = F.logsigmoid(neg_score)
        neg_score -= filter_bias.float()
        pos_score = pos_score.unsqueeze(1)
        rankings = torch.sum(neg_score >= pos_score, dim=1) + 1
        return np.mean(1.0/rankings.cpu().numpy())

Now let's get to the actual training.

We first split the graph into the training set and the testing set. We random sample 80\% edges as the training data and 20\% edges as the testing data. There is no overlap between the training set and the testing set.

In [ ]:
eids = np.random.permutation(g.number_of_edges())
train_eids = eids[:int(len(eids) * 0.8)]
valid_eids = eids[int(len(eids) * 0.8):int(len(eids) * 0.9)]
test_eids = eids[int(len(eids) * 0.9):]
train_g = g.edge_subgraph(train_eids, preserve_nodes=True)

The training loop

In [ ]:
# Model for link prediction
model = LinkPrediction(gconv_model)

# Training hyperparameters
weight_decay = 5e-4
n_epochs = 30
lr = 2e-3
neg_sample_size = 100

# use optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

# initialize graph
dur = []
for epoch in range(n_epochs):
    model.train()
    loss = model(train_g, features, neg_sample_size)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc = LPEvaluate(gconv_model, g, features, valid_eids, neg_sample_size)
    print("Epoch {:05d} | Loss {:.4f} | MRR {:.4f}".format(epoch, loss.item(), acc))

print()
# Let's save the trained node embeddings.
acc = LPEvaluate(gconv_model, g, features, test_eids, neg_sample_size)
print("Test MRR {:.4f}".format(acc))

## Take home exercise

An interested user can try other GNN models to compute node embeddings and use it for node classification. Please check out the [nn module](https://doc.dgl.ai/features/nn.html) in DGL.